In [1]:
import torch
import gym
import gym_mp
import numpy as np
import time
import ray
ray.init()

2020-09-16 16:14:18,107	INFO resource_spec.py:223 -- Starting Ray with 14.7 GiB memory available for workers and up to 7.36 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-09-16 16:14:18,561	INFO services.py:1191 -- View the Ray dashboard at localhost:8265


{'node_ip_address': '192.168.0.10',
 'raylet_ip_address': '192.168.0.10',
 'redis_address': '192.168.0.10:6379',
 'object_store_address': '/tmp/ray/session_2020-09-16_16-14-18_107040_3016/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-09-16_16-14-18_107040_3016/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-09-16_16-14-18_107040_3016'}

In [2]:
gym_mp.iterators.episode('CartPole-v0', mode=gym_mp.mode.sa)

(array([[-0.03570205, -0.03047747,  0.02822699,  0.02090508],
        [-0.0363116 , -0.22599261,  0.02864509,  0.32235861],
        [-0.04083145, -0.42151051,  0.03509226,  0.62393565],
        [-0.04926166, -0.61710438,  0.04757097,  0.92746083],
        [-0.06160375, -0.81283525,  0.06612019,  1.23470558],
        [-0.07786046, -0.61862256,  0.0908143 ,  0.9634483 ],
        [-0.09023291, -0.81483956,  0.11008327,  1.28322367],
        [-0.1065297 , -0.62127809,  0.13574774,  1.02693807],
        [-0.11895526, -0.81792028,  0.1562865 ,  1.35897518],
        [-0.13531367, -1.01461815,  0.18346601,  1.6961874 ],
        [-0.15560603, -1.21132149,  0.21738975,  2.0399275 ]]),
 array([0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0]))

In [3]:

lit = gym_mp.mp_episodes('Pong-v0', workers=2, 
                            mode=gym_mp.mode.sa, n=1, 
                            max_length=1000)

for x in lit.gather_async():
    s, a = x
    print(s.shape, a.shape)
    


(1000, 210, 160, 3) (1000,)
(1000, 210, 160, 3) (1000,)


In [4]:
class TestEnv(gym.Env):
    
    def __init__(self):
        super(TestEnv, self).__init__()
        self.name = "env"
        self.i = 0
        self.n = 3
        self.action_space = gym.spaces.Discrete(2)
    
    def step(self, action):
        self.i += 1
        time.sleep(np.random.randint(1000)/1000)
        done = self.i > self.n
        return "{0}-{1}-{2}".format(self.name, self.i, action), 0., done

    def reset(self):
        self.i = 0
        return "{0}-{1}-{2}".format(self.name, self.i, -1)

In [ ]:

policy_f = lambda : gym_mp.policy.uniform(gym.spaces.Discrete(2))

env_f = TestEnv
lit = gym_mp.async_iterator(env_f, policy_f, repeat=True)

for s in lit:
    print(*s)


env-0--1
env-0--1
env-1-0
env-1-0
env-2-1
env-2-0
env-3-0
env-3-1
env-4-1
env-0--1
env-1-1
env-2-0
env-3-1
env-4-1
env-0--1
env-4-1
env-0--1
env-1-0
env-2-0
env-1-0
env-3-1
env-4-1
env-0--1
env-2-1
env-1-0
env-2-1
env-3-0
env-3-0
env-4-0
env-0--1
env-4-0
env-0--1


In [ ]:
import time
import ray.util.iter

class wait_iter:
    
    def __init__(self, n):
        self.n = n
        self.i = 0
    
    def __next__(self):
        time.sleep(1)
        self.i += 1
        if self.i > self.n:
            raise StopIteration()
        return self.i
    
    def __iter__(self):
        return self
    
def gen(n):
    for i in range(n):
        time.sleep(1)
        yield i

class wait_iter2:
    
    def __init__(self, n):
        self.n = n
    
    def __iter__(self):
        return gen(self.n)
        
    
it = ray.util.iter.from_iterators([range(10), wait_iter2(10)])

lit = it.gather_async()

for i in lit:
    print(i)